In [9]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
%pip install "unstructured[md]" nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 2.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 KB 1.0 MB/s eta 0:00:00:00:01
  Using cached chardet-5.2.0-py3-none-any.whl (199 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 9.4 MB/s eta 0:00:00:00:0100:01
  Using cached wrapt-1.17.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (82 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 7.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 KB 4.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 KB 6.1 MB/s eta 0:00:0000:0

## Document Loading

In [15]:
loader = DirectoryLoader('./manual', glob="**/*.txt", loader_cls=TextLoader, show_progress=True)
docs = loader.load()
print(docs[0])


100%|██████████| 1/1 [00:00<00:00, 1671.04it/s]

page_content='### Behavior Text 
You are an assistant for creating INI files. You can take this INI file as a template: {EXPLANATION_STRING}
This is an INI file enriched with information about the key-value pairs which is inside the angled brackets. In square brackets I have the standard units for the values (if applicable).
The user should either all at once or message by message send values which should be processed by the assistant.
A user may send a message in which he defines one or more key value pairs. Your task is to find and replace the values in the template by the value(s) given in user's message. If you
find in the chat history a previous reply with a full INI file then we continue to update values in this file instead of starting from the template INI file. 
Always reply with the complete (all sections), updated INI file, omitting units and the explanatory text in angled and square brackets. If the user provides keys in German, identify the corresponding English keys in th

## Text Splitting

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=50,
)

split_docs = text_splitter.split_documents(docs)
print(f"Num of chunks: {len(split_docs)}")

Num of chunks: 47


## Embedding Model 

In [ ]:
from langchain_openai import OpenAIEmbeddings
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-large")


## Embed Documents

In [ ]:
from langchain_community.vectorstores import Chroma

CHROMA_PATH = "./chroma_storage"
vectorstore = Chroma.from_documents(documents=split_docs, embedding=embeddings_model, persist_directory=CHROMA_PATH)

In [37]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

question = f"What is the NoOfFlights parameter about?"

PROMPT_TEMPLATE = """
Answer the question based on the following information if neccessary use also your world knowledge of the topic, but keep it concise:

{context}

---

Answer the question based on the following information if neccessary use also your world knowledge of the above topic, but keep it concise: {question}
"""

db = vectorstore

results = db.similarity_search_with_relevance_scores(question, k=5)

context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=question)
print(prompt)

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3)
response_text = llm.invoke(prompt)
print(response_text.content)


/tmp/ipykernel_877101/371347339.py:18: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'source': 'manual/input.txt'}, page_content='NoOfElements=undefined\nNoOfFlights=undefined\nBarrelLength=undefined'), 0.2547724154486646), (Document(metadata={'source': 'manual/input.txt'}, page_content='NoOfFlights=undefined ; Number of flights in the extruder, can use single flight (German: eingängig)\nBarrelLength=undefined ; Length of the barrel/housing, German: Länge, Gehäuselänge or similar , [mm]'), 0.2537904400151164), (Document(metadata={'source': 'manual/input.txt'}, page_content='BarrelStraightCut=undefined ; Depth of the V-cut (default: 2.5 percent of BarrelDiameter, used only for twin screws), user may refer to this as vcut , [mm]\nNoOfElements=undefined ; Number of elements in the extruder'), -0.0994098411036306), (Document(metadata={'source': 'manual/input.txt'}, page_content='[E3DSimulationsettings]\nMeshQuality=undefined ; Mesh resolution, can be: coar

Human: 
Answer the question based on the following information if neccessary use also your world knowledge of the topic, but keep it concise:

NoOfElements=undefined
NoOfFlights=undefined
BarrelLength=undefined

---

NoOfFlights=undefined ; Number of flights in the extruder, can use single flight (German: eingängig)
BarrelLength=undefined ; Length of the barrel/housing, German: Länge, Gehäuselänge or similar , [mm]

---

BarrelStraightCut=undefined ; Depth of the V-cut (default: 2.5 percent of BarrelDiameter, used only for twin screws), user may refer to this as vcut , [mm]
NoOfElements=undefined ; Number of elements in the extruder

---

[E3DSimulationsettings]
MeshQuality=undefined ; Mesh resolution, can be: coarse, medium, fine
HexMesher=undefined ; Mesh generator type: TwinScrew or HollowCylinder
KTPRelease=undefined ; Flag indicating whether KTP release is activated; NO or YES

---

outerdiameter=undefined ; Outer diameter of the screw, German: Minimaler Schneckendurchmesser , [mm